# Selecting Columns with `MetaPanda`

In [1]:
import sys
import numpy as np
import pandas as pd
sys.path.insert(0,"../")
# our main import
import turbopanda as turb

print("turbopanda version: %s" % turb.__version__)

turbopanda version: 0.2.4


## Importing some data

In [2]:
g = turb.read("../data/translation.csv", name="trl")
g

MetaPanda(trl(n=5216, p=11, mem=0.918MB, options=[]))

## Selectors

Unlike traditional `pandas` which is incredibly difficult to access subsets of a DataFrame with ease, we allow the use of `regex` **and** typing (such as `float`) to specify subgroups that contain that capture pattern or data type.

**NOTE**: Using the `__getitem__` attribute of `MetaPanda` **does not alter the underlying `DataFrame`**! The same super-object remains, allowing you to very quickly view dataframe subsets using a selection method of your choice.

The **order of selection** if:

1. selection is `None`: return \[\]
2. selection is of type `pandas.Index`: return those columns
3. selection is an accepted `dtype`: return columns of that dtype
4. selection is callable (i.e function): return columns associated with boolean series
5. selection is of type `str`:
    1. selection is found as `meta_` column and column is of type `bool`
    2. selection is found in `selectors_`
    3. not in `df` column names: use regular expressions (regex)
    4. otherwise selector is column name: return single `Series`

## Viewing

An ability to access the column names you wish to view is an advantage as a pre-stage to selecting the full columns you want.

This is achieved using the `view` command;

In [3]:
g.view("float")

Index(['translation_G1_1', 'translation_G1_2', 'translation_G2M_1',
       'translation_G2M_2', 'translation_MG1_1', 'translation_MG1_2',
       'translation_S_1', 'translation_S_2'],
      dtype='object', name='colnames')

Note in the above example we select by a data type; in this instance, every column with a `dtype=float`, or more specifically, with a `dtype.kind='f'`, when referencing NumPy arrays.

We can also select directly a column of interest:

In [4]:
g.columns

Index(['prot_IDs', 'prot_names', 'Gene_names', 'translation_G1_1',
       'translation_G1_2', 'translation_G2M_1', 'translation_G2M_2',
       'translation_MG1_1', 'translation_MG1_2', 'translation_S_1',
       'translation_S_2'],
      dtype='object', name='colnames')

In [5]:
g.view("prot_names")

Index(['prot_names'], dtype='object', name='colnames')

`turbopanda` also supports regex pattern matching and so attempts to find the next best match if one is not available:

In [6]:
g.view("prot_na")

Index(['prot_names'], dtype='object', name='colnames')

In [7]:
g.view("translation")

Index(['translation_G1_1', 'translation_G1_2', 'translation_G2M_1',
       'translation_G2M_2', 'translation_MG1_1', 'translation_MG1_2',
       'translation_S_1', 'translation_S_2'],
      dtype='object', name='colnames')

In [8]:
import itertools as it

In [9]:
pd.DataFrame(tuple(it.product(('a', 'b', 'c'), ('d'))))

,0,1
0,a,d
1,b,d
2,c,d


NOTE: `g.view` will ALWAYS return a `pandas.Index` object, whether empty or full; thus you can be guaranteed to chain this to other commands related to pandas if you so wish.

Null is also returned:

In [10]:
g.view(None)

Index([], dtype='object', name='colnames')

Using the meta information, boolean-type columns in the meta information can also act as selectors:

In [11]:
g.view("is_mixed_type")

Index(['prot_names', 'Gene_names'], dtype='object', name='colnames')

Viewing every available option as a coded selector can be found in the `options_` attribute:

In [12]:
g.options_

('is_mixed_type', 'is_unique_id')

And finally a more complex example using regex to get exactly the combination of columns you want:

In [13]:
g.view("translation_[G1SM2]+_1")

Index(['translation_G1_1', 'translation_G2M_1', 'translation_MG1_1',
       'translation_S_1'],
      dtype='object', name='colnames')

Or alternatively viewing by a custom function that calls to `DataFrame.apply`.

Here we show any columns whereby the ratio between the number of non-missing values and the full length is the same: i.e the column has no missing values.

In [14]:
g.view(lambda x: x.count()==x.shape[0])

Index(['prot_IDs'], dtype='object', name='colnames')

### Inverse viewing

Similarly, we have the `view_not` function for options which we wish to NOT select for, i.e getting all columns that do not comply with some criterion:

In [15]:
g.view_not(float)

Index(['prot_IDs', 'prot_names', 'Gene_names'], dtype='object', name='colnames')

The same sorts of inputs are applicable in this case also.

## Creating multi-views

By using multiple selection criteria, by default `view` only keeps the **union** of the terms provided:

$$
S=\bigcup_i t_i
$$

This means that if you select for `object` and for "Intensity", you will get all of the column names of type `object` **OR** containing the string "Intensity" within it.

This is contrary to a **intersection** of terms, where you would get the column names of type `object` **AND** they contain the string "Intensity".

In addition, *the order of the elements is maintained*, even across multiple selectors, such that any sorting/order is preserved in future operations.

In [16]:
g.view(float, "_1", "G1")

Index(['translation_G1_1', 'translation_G1_2', 'translation_G2M_1',
       'translation_G2M_2', 'translation_MG1_1', 'translation_MG1_2',
       'translation_S_1', 'translation_S_2'],
      dtype='object', name='colnames')

In the above case, we're selecting columns that are of type float, OR contain `_1`, OR contain `G1` string. 

### Using the intersection

To find the intersect rather than the union of terms, you can use `search`.

**NOTE**: `search` is deprecated and will be removed in version 0.2.7, use `select` instead.

In [17]:
g.search(float, "_1")

..\turbopanda\_deprecator.py:42: FutureWarning: search is deprecated since version 0.2.6, to be removed in version 0.2.8, use function '`MetaPanda.select`' instead, (redundancy with view, view_not.)
  warnings.warn("".join(segments), FutureWarning)


Index(['translation_G1_1', 'translation_G2M_1', 'translation_MG1_1',
       'translation_S_1'],
      dtype='object', name='colnames')

## Using `eval`-like string operations

Any stringable command from the previous calls to `view` and `view_not`, etc can be stacked into a single string and parsed, much like the `pd.eval` function. This can be achieved using the `select` function. 

**WARNING**: Whitespaces are removed and hence no input strings/column names can have spaces in them.

In [20]:
g.select("float & _1$")

Index(['translation_G1_1', 'translation_G2M_1', 'translation_MG1_1',
       'translation_S_1'],
      dtype='object', name='colnames')

In [28]:
g.select("float | _1$ | G1")

Index(['translation_G1_1', 'translation_G1_2', 'translation_G2M_1',
       'translation_G2M_2', 'translation_MG1_1', 'translation_MG1_2',
       'translation_S_1', 'translation_S_2'],
      dtype='object', name='colnames')